In [0]:
%sql
CREATE CATALOG IF NOT EXISTS dev
MANAGED LOCATION 'abfss://unity-catalog-storage@dbstoragehakrjbhbggnca.dfs.core.windows.net/2598701134546467';

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dev.demo_db;


In [0]:
%sql
DROP DATABASE IF EXISTS dev.default CASCADE;

In [0]:
flight_schema_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

flight_time_dt = spark.read.format("csv").schema(flight_schema_ddl).option("dateFormat", "M/d/y").load("abfss://dbfs-container@azuredatabrickstorages.dfs.core.windows.net/external/flight-time.json")


In [0]:
display (flight_time_dt)